In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from DBSCANSupport import DBSCANSupport
from LocalPath import LOCALPATH
from sklearn.cluster import DBSCAN

In [2]:
grav = pd.read_csv(LOCALPATH + 'data/test_grav.csv').drop(columns=["old_ind"])
grav = grav[['Longitude', 'Latitude', 'Intensity']]

In [3]:
Z = DBSCANSupport.formatData(grav, 'Intensity')
#Z = Z[(Z[:, 2] > -20) & (Z[:, 2] < 7)]
data = Z

/Users/seamount_picking_stable/seamount_picking/code/_SeamountSupport.py:264: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["TrueSeamount"] = np.zeros(data.shape[0])


In [4]:
## Elbow test
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt
neighbors = NearestNeighbors(n_neighbors=20)
neighbors_fit = neighbors.fit(data)
distances, indices = neighbors_fit.kneighbors(data)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
elbow = px.scatter(x=range(0, len(distances)), y=distances)#.show()

In [5]:
test_eps = np.linspace(0.1, 0.5, 20)
test_samp = np.arange(2, 10)
DBModel_test = DBSCANSupport(LOCALPATH+"data/sample_mask.txt.xlsx", train_zone=(-6, -1.5, -98, -90))
DBModel_test.addTrainingData(data)

In [6]:
score, params, data_out  = DBModel_test.gridSearch(test_eps, test_samp, verbose=True)

Score for 0.1 and 2 is -7.337748344370861 with 198 clusters
Score for 0.1 and 3 is -7.337748344370861 with 145 clusters
Score for 0.1 and 4 is -7.456953642384106 with 122 clusters
Score for 0.1 and 5 is -5.629139072847682 with 113 clusters
Score for 0.1 and 6 is -5.423841059602649 with 96 clusters
Score for 0.1 and 7 is -5.258278145695364 with 86 clusters
Score for 0.1 and 8 is -4.516556291390729 with 71 clusters
Score for 0.1 and 9 is -4.066225165562914 with 59 clusters
Score for 0.12105263157894737 and 2 is -4.52317880794702 with 120 clusters
Score for 0.12105263157894737 and 3 is -4.52317880794702 with 85 clusters
Score for 0.12105263157894737 and 4 is -4.629139072847682 with 78 clusters
Score for 0.12105263157894737 and 5 is -5.013245033112582 with 66 clusters
Score for 0.12105263157894737 and 6 is -3.0728476821192054 with 56 clusters
Score for 0.12105263157894737 and 7 is -2.9536423841059603 with 49 clusters
Score for 0.12105263157894737 and 8 is -3.3642384105960264 with 50 cluste

In [ ]:
dfout = pd.DataFrame(data_out, columns=["Lat", "Lon", "Label", "Intensity", "TrueLabel"])
df_labeled = dfout[dfout['Label'] != 0]

In [ ]:
fig = px.scatter(df_labeled, x="Lon", y="Lat", color="Label")
fig.show()

In [ ]:
fig2 = px.scatter(dfout, x="Lon", y="Lat", color="TrueLabel")
fig2.update_xaxes(
    scaleanchor="y",
    scaleratio=1,
  )

In [ ]:
with open(LOCALPATH + "out/best.txt", "a") as f:
    f.write("Score: " + str(score) + "\n")
    f.write("Params: " + str(params) + "\n")
DBModel_test.matchPoints().to_csv(LOCALPATH + "out/matched.csv", index=False)
dfout.to_csv(LOCALPATH + "out/labels.csv", index=False)
